In [28]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import pandas as pd
from pprint import pprint
from PIL import Image
import urllib

In [2]:
mars_news_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [3]:
news_response = requests.get(mars_news_url)

In [4]:
news_soup = bs(news_response.text, 'html.parser') 

In [5]:
# print(news_soup.prettify()) 

In [6]:
news_results = news_soup.find_all('div', class_="slide") 

In [7]:
# news_results

In [8]:
for result in news_results:
    try:
        news_title = result.find('div', class_="content_title").text
        news_p = result.a.div.text

        if (news_title and news_p):
            print(f'-------------{news_title}\n{news_p}')
    except AttributeError as e:
        print(e)

-------------

NASA's Curiosity Mars Rover Finds a Clay Cache




The rover recently drilled two samples, and both showed the highest levels of clay ever found during the mission.





-------------

Why This Martian Full Moon Looks Like Candy




For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.





-------------

NASA Garners 7 Webby Award Nominations




Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.





-------------

NASA's Opportunity Rover Mission on Mars Comes to End




NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.





-------------

NASA's InSight Places First Instrument on Ma

In [9]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False) 

In [10]:
image_url= "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(image_url) 

In [11]:
image_html = browser.html
image_soup = bs(image_html, 'html.parser')

featured_image = image_soup.find('article', class_='carousel_item')["style"]

link_featured_image = featured_image.split("'")[1]

base_url = "https://www.jpl.nasa.gov"

featured_image_url = base_url + link_featured_image
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18087-1920x1200.jpg'

In [12]:
weather_url = "https://twitter.com/MarsWxReport"
weather_response = requests.get(weather_url)
weather_soup = bs(weather_response.text, 'html.parser') 
print(weather_soup.prettify()) 

<!DOCTYPE html>
<html data-scribe-reduced-action-queue="true" lang="en">
 <head>
  <meta charset="utf-8"/>
  <script nonce="7Hm8wo8ROt81P5CWpKs93w==">
   !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();
  </script>
  <script id="bouncer_terminate_iframe" nonce="7Hm8wo8ROt81P5CWpKs93w==">
   if (window.top != window) {
  window.top.postMessage({'bouncer': true, 'event': 'complete'}, '*');
}
  </script>
  <script id="swift_action_queue" nonce="7Hm8wo8ROt81P5CWpKs93w==">
   !function(){function e(e){if(e||(e=window.event),!e)return!1;if(e.timestamp=(new Date).getTime(),!e.target&&e.srcElement&&(e.target=e.srcElement),document.documentElement.getAttribute("data-scribe-reduced-action-queue"))for(var t=e.target;t&&t!=document.body;){if("A"==t.tagName)return;t=t.parentNode}return i("all",o(e)),a(e)?(document.addEventListener||(e=o(

In [13]:
mars_weather = weather_soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 261 (2019-08-21) low -102.4ºC (-152.4ºF) high -26.6ºC (-15.8ºF)\nwinds from the SSE at 4.9 m/s (11.0 mph) gusting to 16.0 m/s (35.8 mph)\npressure at 7.70 hPapic.twitter.com/MhPPOHJg3m'

In [14]:
facts_url = "https://space-facts.com/mars/"
facts_response = requests.get(facts_url)
facts_soup = bs(facts_response.text, 'html.parser') 
print(facts_soup.prettify()) 

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://space-facts.com/wp/xmlrpc.php" rel="pingback"/>
  <script>
   function fvmuag(){if(navigator.userAgent.match(/x11.*fox\/54|oid\s4.*xus.*ome\/62|oobot|ighth|tmetr|eadles|ingdo/i))return!1;if(navigator.userAgent.match(/x11.*ome\/75\.0\.3770\.100/i)){var e=screen.width,t=screen.height;if("number"==typeof e&&"number"==typeof t&&862==t&&1367==e)return!1}return!0}
  </script>
  <title>
   Mars Facts - Interesting Facts about Planet Mars
  </title>
  <!-- Jetpack Site Verification Tags -->
  <meta content="OI26qBRJ2SI673vDUn_KgAd_H89WIX3IEgf6TZbhgDs" name="google-site-verification">
   <meta content="988097676DFC89A04403B1F262C0F731" name="msvalidate.01">
    <!-- This site is optimized with the Yoast SEO plugin v11.9 - https://yoast.com/wordpress/plugins/seo/ -->
    <link hre

In [15]:
fact_table= pd.read_html(facts_url)[1]
fact_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
facts = fact_table.to_html(header=False, index=False, index_names=False)
pprint(facts)

('<table border="1" class="dataframe">\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <td>Equatorial Diameter:</td>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Polar Diameter:</td>\n'
 '      <td>6,752 km</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Mass:</td>\n'
 '      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Moons:</td>\n'
 '      <td>2 (Phobos &amp; Deimos)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Orbit Distance:</td>\n'
 '      <td>227,943,824 km (1.38 AU)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Orbit Period:</td>\n'
 '      <td>687 days (1.9 years)</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Surface Temperature:</td>\n'
 '      <td>-87 to -5 °C</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>First Record:</td>\n'
 '      <td>2nd millennium BC</td>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <td>Recorded By:</td>\n'
 '      <td>Egyptian astronomers</td>\n'
 '    </tr>\n'
 '  </tbody>\n'
 '</tabl

In [17]:
hemisphere_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url) 

In [18]:
hemisphere_html = browser.html
hemisphere_soup = bs(hemisphere_html, 'html.parser')

new_image = hemisphere_soup.find_all("a", class_="itemLink product-item")

hemisphere_base_url = "https://astrogeology.usgs.gov"

new_image

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png"/></a>,

In [19]:
image_sources = []
image_url_list = []
titles = []
iteration = 0

for links in new_image:
    end_url = links.get("href")
    if end_url not in image_sources:
        image_sources.append(end_url)
        iteration += 1
        click_url = hemisphere_base_url + end_url
        browser.visit(click_url)
        titles.append(browser.find_by_tag('h2').text)        
        browser.find_link_by_text("Sample").first.click()
        print(browser.find_link_by_text("Sample"))
        print(browser.title)
        url_for_image = browser.windows[iteration].url
        image_url_list.append(url_for_image)

image_url_list

Cerberus Hemisphere Enhanced | USGS Astrogeology Science Center
Schiaparelli Hemisphere Enhanced | USGS Astrogeology Science Center
Syrtis Major Hemisphere Enhanced | USGS Astrogeology Science Center
Valles Marineris Hemisphere Enhanced | USGS Astrogeology Science Center


['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [20]:
pprint(titles)
pprint(image_url_list)

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']
['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']


In [21]:
hemisphere_image_urls = [
    {"title":titles[0], "img_url":image_url_list[0]},
    {"title":titles[1], "img_url":image_url_list[1]},
    {"title":titles[2], "img_url":image_url_list[2]},
    {"title":titles[3], "img_url":image_url_list[3]}
]

In [41]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [44]:
i = 0
for picture in image_url_list:
    Picture_request = requests.get(picture)
    if Picture_request.status_code == 200:
        with open(f"{titles[i]}.jpg", 'wb') as new:
            new.write(Picture_request.content)
            i = i + 1

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'